In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable

In [2]:
transform = transforms.Compose([transforms.RandomResizedCrop(24), transforms.ToTensor()])


train = datasets.FashionMNIST(root="FashionMNIST/processed/", train=True, download=True, transform=transform)
test = datasets.FashionMNIST(root="FashionMNIST/processed/", train=False, download=True, transform=transform)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting FashionMNIST/processed/FashionMNIST\raw\train-images-idx3-ubyte.gz to FashionMNIST/processed/FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting FashionMNIST/processed/FashionMNIST\raw\train-labels-idx1-ubyte.gz to FashionMNIST/processed/FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting FashionMNIST/processed/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to FashionMNIST/processed/FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting FashionMNIST/processed/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to FashionMNIST/processed/FashionMNIST\raw



c:\users\91832\appdata\local\programs\python\python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
test

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: FashionMNIST/processed/
    Split: Test
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(24, 24), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               ToTensor()
           )

In [4]:
batch_size=32
train_data = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
test_data = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=True)

In [22]:
next(iter(train_data))[0].view(32,-1).shape[-1]

576

In [39]:
in_features=next(iter(train_data))[0].view(32,-1).shape[-1]
in_features

576

In [40]:
class Net(nn.Module):
    def __init__(self,in_features):
        super().__init__()
        self.fc1=nn.Linear(in_features,128)
        self.fc2=nn.Linear(128,128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4=nn.Linear(32,10)
        self.relu=nn.ReLU()
        self.softmax=nn.Softmax()
        self.dropout=nn.Dropout(0.2)
        
    def forward(self,x):
        x=self.relu(self.fc1(x))
        x=self.dropout(x)
        x=self.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.relu(self.fc3(x))
        x=self.dropout(x)
        x=self.softmax(self.fc4(x))
        
        return x

In [43]:
model=Net(in_features)
criterion=nn.CrossEntropyLoss()
# from torch.optim.lr_scheduler import StepLR
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
# scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [44]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

training_loss_epoch=[]
val_loss_epoch=[]

best_weights=dict()

epochs=50

model.to(device)
criterion.to(device)

#Training the model
for i in range(epochs):
    train_loss=0
    val_loss=0
    total_corrects=0
    
    #Training model
    model.train()
    for images,labels in train_data:
        
        optimizer.zero_grad()
        
        images=images.view(32,-1) #flattening the image
        
#         images= images.type(torch.float)
#         labels= labels.type(torch.float)
        
        images.to(device) #uploading images and labels to the device
        labels.to(device)
        
        output=model(images) #output is our model predictions
        loss=criterion(output,labels) #calculating the loss
        
        loss.backward() #applying backward propagation
        
        optimizer.step() #updating the parameters(Gradient Descent)
#         scheduler.step()
        
        train_loss+=loss
    training_loss_epoch.append(train_loss)
    
    #Evaluation Phase
    model.eval() #no drop out here 
    size=len(test_data.dataset)
    
    with torch.no_grad(): #context manager why because we wont apply gradients in validation phase
        no_of_batches=size//batch_size
        total_val_loss=0
        correct_pred = 0
        for images,labels in test_data:
            
            images=images.view(32,-1) #flattening the image
        
#             images=images.type(torch.float)
#             labels=labels.type(torch.float)
        
            images.to(device) #uploading images and labels to the device
            labels.to(device)
            
            logits=model(images)
            val_loss_=criterion(logits,labels)
            
            val_loss+=val_loss_
            
            pred = torch.max(logits,1).indices
            
            cnt = torch.eq(labels,pred).sum().item()

            
            correct_pred += cnt
            
    total_corrects += correct_pred        
    
            
    print(f"Running the epoch{i}/{epochs}")
    #print('\n')
    print('__' * 20)
    print(f'Training loss: {train_loss}')
    print(f'Validation loss: {val_loss}')
    print(f'Accuracy score: {total_corrects/size}\n')
    
        

<ipython-input-40-3ebabfa07fce>:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(self.fc4(x))


Running the epoch0/50
________________________________________
Training loss: 3808.296142578125
Validation loss: 602.5248413085938
Accuracy score: 0.5223

Running the epoch1/50
________________________________________
Training loss: 3626.6826171875
Validation loss: 595.496826171875
Accuracy score: 0.5476

Running the epoch2/50
________________________________________
Training loss: 3586.671142578125
Validation loss: 587.8648681640625
Accuracy score: 0.5748

Running the epoch3/50
________________________________________
Training loss: 3565.688232421875
Validation loss: 585.3892822265625
Accuracy score: 0.5824

Running the epoch4/50
________________________________________
Training loss: 3557.0419921875
Validation loss: 586.276611328125
Accuracy score: 0.5769

Running the epoch5/50
________________________________________
Training loss: 3545.450927734375
Validation loss: 581.3710327148438
Accuracy score: 0.5928

Running the epoch6/50
________________________________________
Training loss

KeyboardInterrupt: 